
Este código utiliza a API do Kaggle para baixar o dataset **"Olist Brazilian E-Commerce"** diretamente para o diretório especificado no computador. O dataset é extraído e salvo no caminho definido.


In [ ]:
import kaggle
import os

# Nome do dataset extarido do Kaggle
dataset_name = "olistbr/brazilian-ecommerce"

# Define o diretório onde salvar
save_path = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw"

# Baixa o dataset, usando a api
kaggle.api.dataset_download_files(dataset_name, path=save_path, unzip=True)

print("Download concluído! Arquivos salvos em:", save_path)


Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
Download concluído! Arquivos salvos em: C:\GitHub\Projeto Power bi\data\Kaggle


In [ ]:
# Lista os arquivos no diretório
files = os.listdir(save_path)

# Exibe os arquivos
print("Arquivos no diretório:", save_path)
for file in files:
    print(file)

Arquivos no diretório: C:\GitHub\Projeto Power bi\data\Kaggle
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


In [10]:
import os

# Diretório onde os arquivos estão localizados
directory = r"C:\GitHub\Projeto-Power-BI-Data-Analysis\data\raw"

# Listar todos os arquivos no diretório
for filename in os.listdir(directory):
    # Verificar se é um arquivo CSV
    if filename.endswith(".csv"):
        # Criar o novo nome com o prefixo "stg_"
        new_name = "stg_" + filename
        
        # Obter o caminho completo dos arquivos
        old_file = os.path.join(directory, filename)
        new_file = os.path.join(directory, new_name)
        
        # Renomear o arquivo
        os.rename(old_file, new_file)
        print(f"Renomeado: {filename} -> {new_name}")


Renomeado: olist_customers_dataset.csv -> stg_olist_customers_dataset.csv
Renomeado: olist_geolocation_dataset.csv -> stg_olist_geolocation_dataset.csv
Renomeado: olist_orders_dataset.csv -> stg_olist_orders_dataset.csv
Renomeado: olist_order_items_dataset.csv -> stg_olist_order_items_dataset.csv
Renomeado: olist_order_payments_dataset.csv -> stg_olist_order_payments_dataset.csv
Renomeado: olist_order_reviews_dataset.csv -> stg_olist_order_reviews_dataset.csv
Renomeado: olist_products_dataset.csv -> stg_olist_products_dataset.csv
Renomeado: olist_sellers_dataset.csv -> stg_olist_sellers_dataset.csv
Renomeado: product_category_name_translation.csv -> stg_product_category_name_translation.csv
